In [7]:
import random
import operator
import torch

import latexify


In [8]:
ops = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': operator.truediv
}

def safe_div(x, y):
    if y == 0:
        return float('inf')
    else:
        return x / y


In [9]:
class Individual:
    def __init__(self, gene_length=5):
        self.gene_length = gene_length
        self.genes = [(random.randint(1, 5), random.choice(list(ops.keys()))) for _ in range(gene_length - 1)]
        self.genes.append((random.randint(1, 5), ''))  # end with a number
        self.fitness = float('inf')

    def calculate_fitness(self, target):
        result = self.genes[0][0]
        for num, op in self.genes[1:]:
            if op == '/':
                result = safe_div(result, num)
            elif op:
                result = ops[op](result, num)
        self.fitness = abs(target - result)

    def mutate(self):
        mutation_index = random.randint(0, self.gene_length - 1)
        self.genes[mutation_index] = (random.randint(1, 5), self.genes[mutation_index][1])


In [10]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1.genes) - 1)
    child1_genes = parent1.genes[:crossover_point] + parent2.genes[crossover_point:]
    child2_genes = parent2.genes[:crossover_point] + parent1.genes[crossover_point:]
    return Individual(child1_genes), Individual(child2_genes)

def evolve(population, target):
    new_generation = []
    for _ in range(len(population) // 2):
        parent1, parent2 = random.sample(population, 2)
        child1, child2 = crossover(parent1, parent2)
        child1.mutate()
        child2.mutate()
        child1.calculate_fitness(target)
        child2.calculate_fitness(target)
        new_generation.extend([child1, child2])
    return new_generation


In [11]:
@latexify.with_latex
def evaluate_expression(expr):
    """
    Evaluates an arithmetic expression represented as a list of (number, operator) tuples.
    The first tuple should have an empty string as the operator.
    
    Args:
        expr (list): List of tuples (number, operation) to be evaluated.

    Returns:
        LaTeX representation of step-by-step evaluation.

    Example input:
    [(5, ''), (3, '+'), (2, '*'), (0, '/'), (4, '-')]
    """
    result = expr[0][0]  # Initialize result with the first number
    for num, op in expr[1:]:  # Iterate over the rest of the tuples
        if op:  # Ensure there is an operation to perform
            func = ops[op]  # Get the operator function from the dictionary
            if op == '/':  # Check if the operation is division
                result = safe_div(result, num)  # Perform safe division
            else:
                result = func(result, num)  # Perform the operation
    return result

# Example usage to generate LaTeX
expression = [(5, ''), (3, '+'), (2, '*'), (0, '/'), (4, '-')]
latex_code = evaluate_expression(expression)
print(latex_code)



AttributeError: module 'latexify' has no attribute 'with_latex'